In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from IPython.terminal.shortcuts.filters import PassThrough
from sklearn.impute import SimpleImputer
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix , ConfusionMatrixDisplay
import matplotlib.pyplot as plt



df = pd.read_csv("train.csv")

df.head(100)
drop = ['Cabin','Name','PassengerId','Ticket'] #Virer les colonnes qui servent à rien
df = df.drop(columns = drop)

num_cols = [c for c in df.columns if is_numeric_dtype(df[c])] #Trouver les colonnes numériques
other_cols = [c for c in df.columns if not is_numeric_dtype(df[c])] #Trouver les autres colonnes

OH_encoder = OneHotEncoder(sparse_output= False) #Initialisation de l'OH encoder

for col in other_cols : #On encode les chaines de caractères avec OH car peu de catégories
    encoded_features = OH_encoder.fit_transform(df[[col]])
    encoded_df = pd.DataFrame(encoded_features, columns=OH_encoder.get_feature_names_out([col]))
    df = pd.concat([df, encoded_df], axis=1).drop(columns=[col])

cols_with_na = [c for c in num_cols if df[c].isna().any()] #Trouver les NaN
imputer = SimpleImputer(strategy='mean') #On va remplacer les NaN par la moyenne de la colonne
df[cols_with_na] = imputer.fit_transform(df[cols_with_na])

# Séparation des features (X) et de la target (Y)
Y = df['Survived']
X = df.drop(columns='Survived')

# Vérification des formes
print("Forme de X :", X.shape)
print("Forme de Y :", Y.shape)

# Séparation en train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

param_grid = { #Initialisation des différents hyperparamètres
    'n_estimators' : [50,100,150,200],
    'max_depth' : [3, 5, 7],
    'learning_rate' : [0.01 , 0.1 , 0.2, 0.3],
    'min_child_weight' : [1 , 2 , 3 , 5],
    'subsample' : np.arange(0,1,0.1).tolist(),
    'colsample_bytree' : np.arange(0,1,0.1).tolist(),
    'gamma' : np.arange(0,1,0.1).tolist(),
    'reg_alpha' : np.arange(0,5,0.1).tolist(),
    'reg_lambda' : np.arange(0,5,0.1).tolist(),
    'scale_pos_weight' : np.arange(0,10,1).tolist()

}

model = xgb.XGBClassifier(random_state=42)

grid_search = GridSearchCV( #On cherche les meileurs combinaisons d'hyperparamètres
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1
)

grid_search.fit(X_train,Y_train)

best_model = grid_search.best_estimator_ #On choisit le meilleur
print("Les meilleurs hyperparamètres sont : ", grid_search.best_params_)

Y_pred = best_model.predict(X_test)
print(classification_report(Y_test,Y_pred))

#Evaluation du modèle

accuracy = accuracy_score(Y_test,Y_pred) #Calcul de l'accuracy du modèle
print("L'accuracy-score : ", accuracy)



Forme de X : (891, 11)
Forme de Y : (891,)
